In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## Basic App: Question & Answering from a Document

In [2]:
from langchain_openai import OpenAI

In [3]:
llm = OpenAI()

**Load the text file**

In [4]:
from langchain.document_loaders import TextLoader

In [5]:
loader = TextLoader("data/be-good-and-how-not-to-die.txt")

In [6]:
document = loader.load()

**The document is loaded as a Python list with metadata**

In [7]:
print(type(document))

<class 'list'>


In [8]:
print(len(document))

1


In [9]:
print(document[0].metadata)

{'source': 'data/be-good-and-how-not-to-die.txt'}


In [10]:
print(f"You have {len(document)} document.")

You have 1 document.


In [11]:
print(f"Your document has {len(document[0].page_content)} characters")

Your document has 27423 characters


**Split the document in small chunks**

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=400
)

In [14]:
document_chunks = text_splitter.split_documents(document)

In [15]:
print(f"Now you have {len(document_chunks)} chunks.")

Now you have 12 chunks.


**Convert text chunks in numeric vectors (called "embeddings")**

In [16]:
from langchain.embeddings.openai import OpenAIEmbeddings

In [17]:
embeddings = OpenAIEmbeddings()

c:\users\johnd\appdata\local\programs\python\python39\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


**Load the embeddings to a vector database**

In [18]:
from langchain.vectorstores import FAISS

*Careful: the next operation is expensive in OpenAI*

In [19]:
stored_embeddings = FAISS.from_documents(document_chunks, embeddings)

**Create a Retrieval Question & Answering Chain**

In [20]:
from langchain.chains import RetrievalQA

In [21]:
QA_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=stored_embeddings.as_retriever()
)

**Now we have a Question & Answering APP**

In [22]:
question = """
What is this article about? 
Describe it in less than 100 words.
"""

In [23]:
QA_chain.run(question)

/Users/juliocolomer/.pyenv/versions/3.11.4/envs/venv020124/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' This article is about the importance of persistence and determination in the face of challenges and setbacks in the startup world. The author uses examples of successful and unsuccessful startups to illustrate the idea that avoiding failure is the key to achieving success. The article also explores the role of fear of failure in motivating founders to persevere through difficult times. Ultimately, the article encourages readers to not give up when facing obstacles and to stay committed to their goals in order to increase their chances of success.'

In [24]:
question2 = """
And how does it explain how to create somethin people want?
"""

In [25]:
QA_chain.run(question2)

' The context provided does not explicitly explain how to create something people want. However, the essay does mention some key points that could be useful in understanding what people want. These include the importance of focusing on users and their needs, the value of solving a problem or filling a gap in the market, and the role of benevolence and helping others in driving motivation and success. Ultimately, creating something people want may involve a combination of these factors and can vary depending on the specific product or service being offered.'

## Connect RAG App with FastAPI

In [28]:
#!pip install fastapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 2.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [29]:
from fastapi import FastAPI, HTTPException

In [30]:
app = FastAPI()

In [31]:
@app.post("/conversation")
async def conversation(query: str):
    try:
        result = QA_chain.run(query=query)
        return {"response": result}
    except Exception as e:
        raise HTTPException(detail=str(e), status_code=500)

In [34]:
#!pip install uvicorn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 980.5 kB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [35]:
import uvicorn

In [36]:
config = uvicorn.Config(app, host="0.0.0.0", port=5566)
server = uvicorn.Server(config)

In [ ]:
await server.serve()

INFO:     Started server process [38268]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:5566 (Press CTRL+C to quit)


* Open http://0.0.0.0:5566/docs#
* In Windows, open http://localhost:5566/docs#

**If you are using VS Code editor, use the following instead and then execute python filename.py in your terminal:**

In [ ]:
# if __name__ == "__main__":
#     import uvicorn

#     uvicorn.run(app, host="0.0.0.0", port=5566)